## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [28]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.
# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

img = cv2.imread(images[0])
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#print(len(images))

#Step through the list and search for chessboard corners
for fname in images:
    #print(fname)
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        # Writing the output images to the output_images folder
        #cv2.imwrite("output_images/"+fname+"_CCoutput"+".jpg",img)

## Calibrate the camera and get the Camera Matrix and Distortion co-efficient

In [2]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [3]:
def cal_undistort(img, mtx, dist):    
    # Use cv2.undistort()    
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

## Checking distortion correction on test_images

In [4]:
images = glob.glob('test_images/test*.jpg')
for each in images:
    img = cv2.imread(each)
    dst = cal_undistort(img,mtx,dist)
    # Saving the undistorted images to the output dir
    cv2.imwrite("output_images/"+each+"_Undistorted"+".jpg",dst)
images = glob.glob('test_images/straight_lines*.jpg')
for each in images:
    img = cv2.imread(each)
    dst = cal_undistort(img,mtx,dist)
    #Saving the undistorted images to the output dir
    cv2.imwrite("output_images/"+each+"_Undistorted"+".jpg",dst)    


## Carrying out Perspective Transform on One Undistorted test image

In [49]:
img = mpimg.imread('output_images/test_images/test5.jpg_Undistorted.jpg')
plt.imshow(img)

img_size = (img.shape[1], img.shape[0])
#src = np.float32([[604,447],[677,447] ,[200,720],[1090,720]])
# TO-DO May be you need to decrease the height of the trapezoid
#src = np.float32([[541,492],[750,492] ,[200,720],[1096,720]])
src = np.float32([[467,532],[822,532] ,[171,720],[1125,720]])
dst = np.float32([[200,400],[1100,400] ,[200,720],[1100,720]])

def do_perspective_transform(img):
    M = cv2.getPerspectiveTransform(src,dst)
    warped = cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(warped)
    ax2.set_title('Undistorted and Warped Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    return warped


## Carrying out Perspective Transform on all Undistorted test images

In [51]:
images = glob.glob('output_images/test_images/test*.jpg')
for each in images:
    img = mpimg.imread(each)
    do_perspective_transform(img)

## Function to calculate the gradient along a given direction

In [5]:
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if(orient=='x'):
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    mask = (scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[mask]=1
    # 6) Return this mask as your binary_output image
    
    return sxbinary

## Function to calculate the magnitude of the gradient

In [6]:
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    thresh_min = mag_thresh[0]
    thresh_max = mag_thresh[1]
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    # 3) Calculate the magnitude 
    abs_sobelxy = np.sqrt(np.power(sobel_x,2)+np.power(sobel_y,2))
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    # 5) Create a binary mask where mag thresholds are met
    mask = (scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[mask]=1
    # 6) Return this mask as your binary_output image
    
    return sxbinary

## Function to calculate the direction of the gradient

In [7]:
def dir_threshold(img, sobel_kernel, thresh):
    thresh_min = thresh[0]
    thresh_max = thresh[1]
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobel_x = np.absolute(sobel_x)
    abs_sobel_y = np.absolute(sobel_y)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    dirn_gradient = np.arctan2(abs_sobel_y, abs_sobel_x)
    # 5) Create a binary mask where direction thresholds are met
    mask = (dirn_gradient >= thresh_min) & (dirn_gradient <= thresh_max)
    # 6) Return this mask as your binary_output image
    sxbinary = np.zeros_like(dirn_gradient)
    sxbinary[mask]=1
    return sxbinary

## Visualizing Sobel thresholds.
**We shall first run the thresholds on one test_image and when we fine tune the thresholds**
**we shall then check the thresholds on all test images**

##  Absolute sobel threshold

In [15]:
from ipywidgets import interact, interactive, fixed
def sobel_update(thresh_min=20, thresh_max=200):
    test_image = cv2.imread("output_images/test_images/test5.jpg_Undistorted.jpg")
    dst = cal_undistort(test_image,mtx,dist)
    output_img = abs_sobel_thresh(dst, 'x', thresh_min, thresh_max)
    # Visualize sobel absolute threshold
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    f.subplots_adjust(hspace = .2, wspace=.05)
    ax1.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
    ax1.set_title('Unwarped Image', fontsize=30)
    ax2.imshow(output_img, cmap='gray')
    ax2.set_title('Sobel Absolute', fontsize=30)

interact(sobel_update, 
         thresh_min=(0,255), 
         thresh_max=(0,255))

print('...')

interactive(children=(IntSlider(value=20, description='thresh_min', max=255), IntSlider(value=200, description…

...


## Visualizing Colorspaces thresholding
[Colorspaces for lane detection](https://towardsdatascience.com/teaching-cars-to-see-advanced-lane-detection-using-computer-vision-87a01de0424f)
***
**Using information in the above link we shall attempt only the following thresholds**.
**We shall first run the thresholds on one test_image and when we fine tune the thresholds**
**we shall then check the thresholds on all test images**

## R channel Thresholding

In [42]:
def R_threshold(image,thresh=(0, 255)):
    R = image[:,:,0]
    G = image[:,:,1]
    B = image[:,:,2]
    binary = np.zeros_like(R)
    binary[(R > thresh[0]) & (R <= thresh[1])] = 1
    return binary

## HLS S-channel Thresholding

In [48]:
def S_threshold(image,thresh=(0, 255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    binary = np.zeros_like(H)
    binary[(S > thresh[0]) & (S <= thresh[1])] = 1
    return binary

## HLS L-channel Thresholding

In [51]:
def L_threshold(image,thresh=(0, 255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    binary = np.zeros_like(H)
    binary[(L > thresh[0]) & (L <= thresh[1])] = 1
    return binary

## HSV V-channel Thresholding

In [52]:
def V_threshold(image,thresh=(0, 255)):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    H = hsv[:,:,0]
    S = hsv[:,:,1]
    V = hsv[:,:,2]
    binary = np.zeros_like(H)
    binary[(V > thresh[0]) & (V <= thresh[1])] = 1
    return binary

## LAB L-channel Thresholding

In [53]:
def Lab_L_threshold(image,thresh=(0, 255)):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
    L = lab[:,:,0]
    A = lab[:,:,1]
    B = lab[:,:,2]
    binary = np.zeros_like(H)
    binary[(L > thresh[0]) & (L <= thresh[1])] = 1
    return binary

## LAB B-channel Thresholding

In [54]:
def B_threshold(image,thresh=(0, 255)):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
    L = lab[:,:,0]
    A = lab[:,:,1]
    B = lab[:,:,2]
    binary = np.zeros_like(H)
    binary[(B > thresh[0]) & (B <= thresh[1])] = 1
    return binary

In [ ]:
test_image = cv2.imread("output_images/test_images/test5.jpg_Undistorted.jpg")
def color_update(min_thresh, max_thresh):
    output_img_R = R_threshold(image,(min_thresh, max_thresh))
    output_img_S = S_threshold(image,(min_thresh, max_thresh))
    output_img_L = L_threshold(image,(min_thresh, max_thresh))
    output_img_V = V_threshold(image,(min_thresh, max_thresh))
    output_img_Lab_L = Lab_L_threshold(image,(min_thresh, max_thresh))
    output_img_B = B_threshold(image,(min_thresh, max_thresh))    
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    f.subplots_adjust(hspace = .2, wspace=.05)
    
    # Visualize R-channel threshold
    ax1.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
    ax1.set_title('Undistorted Image', fontsize=30)
    ax2.imshow(output_img_R, cmap='gray')
    ax2.set_title('RGB R-Channel', fontsize=30)
    
    # Visualize S-channel threshold
    ax1.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
    ax1.set_title('Undistorted Image', fontsize=30)
    ax2.imshow(output_img_S, cmap='gray')
    ax2.set_title('HLS S-Channel', fontsize=30)
    
    # Visualize L-channel threshold
    ax1.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
    ax1.set_title('Undistorted Image', fontsize=30)
    ax2.imshow(output_img_L, cmap='gray')
    ax2.set_title('HLS L-Channel', fontsize=30)

interact(update,
         min_thresh=(0,255), 
         max_thresh=(0,255))

print('...')